<a href="https://colab.research.google.com/github/arunm917/CS6910_Assignment_3/blob/main/CS6910_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv
import gdown
import pandas as pd

In [3]:
# downloading file from gdrive
output = 'tam_train'
file_id = '1pdJVD8P71fpqGRnvFfOp_6TbVft9NlnH' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('DONE.')

Downloading...
From: https://drive.google.com/uc?id=1pdJVD8P71fpqGRnvFfOp_6TbVft9NlnH
To: /content/tam_train
100%|██████████| 2.69M/2.69M [00:00<00:00, 61.9MB/s]


DONE.


In [5]:
train_data_df = pd.read_csv('tam_train')

In [7]:
train_data_df.columns = ['English','Tamil']

In [8]:
train_data_df

,English,Tamil
0,menmaithaan,மென்மைதான்
1,avarantri,அவரன்றி
2,mudiyarathu,முடியறது
3,aadaiyanigalaal,ஆடையணிகளால்
4,muzhumaiyaakkugindrana,முழுமையாக்குகின்றன
...,...,...
51194,melakkannicherry,மேலக்கன்னிச்சேரி
51195,thattippaarththaalum,தட்டிப்பார்த்தாலும்
51196,meribaavin,மெரிபாவின்
51197,janmath,ஜன்மத்


In [20]:
[*train_data_df.loc[0, 'Tamil']]

['ம', 'ெ', 'ன', '்', 'ம', 'ை', 'த', 'ா', 'ன', '்']

In [21]:
len(train_data_df['English'])

51199

In [63]:
# Creating vocabulary

vocabulary = []
for i in range(len(train_data_df['English'])):
  char = [*train_data_df.loc[i, 'English']]
  vocabulary.extend(char)

for i in range(len(train_data_df['Tamil'])):
  char = [*train_data_df.loc[i, 'Tamil']]
  vocabulary.extend(char)
vocabulary = set(vocabulary)
print(len(vocabulary))

72


In [64]:
print(vocabulary)

{'n', 'ஒ', 'h', 'z', 'ோ', 'v', 'வ', 'ச', 'ந', 'ஜ', 'ய', 'u', 'ன', 'a', 'f', 'y', 'r', 'p', 'w', 'ஆ', 't', 'k', 'இ', 'e', 'g', 'ஷ', 'c', 'ஞ', 'ஊ', 'ட', 'b', 'ஓ', 'ே', 'i', 'ம', 'ர', 'ஹ', 'த', 'd', 'ு', 'l', 'o', 'ா', 'ற', 'ண', 'ல', 'எ', 'ஃ', 'ி', 'ௌ', 'x', 'ை', 'j', 'அ', 'ூ', 's', 'ீ', 'm', 'ள', 'ஸ', 'ஏ', 'க', 'ொ', 'ெ', 'ழ', 'உ', 'q', '்', 'ஐ', 'ஈ', 'ங', 'ப'}


In [65]:
# Indexing

char_index = {value: index for index, value in enumerate(vocabulary)}
print(char_index)

{'n': 0, 'ஒ': 1, 'h': 2, 'z': 3, 'ோ': 4, 'v': 5, 'வ': 6, 'ச': 7, 'ந': 8, 'ஜ': 9, 'ய': 10, 'u': 11, 'ன': 12, 'a': 13, 'f': 14, 'y': 15, 'r': 16, 'p': 17, 'w': 18, 'ஆ': 19, 't': 20, 'k': 21, 'இ': 22, 'e': 23, 'g': 24, 'ஷ': 25, 'c': 26, 'ஞ': 27, 'ஊ': 28, 'ட': 29, 'b': 30, 'ஓ': 31, 'ே': 32, 'i': 33, 'ம': 34, 'ர': 35, 'ஹ': 36, 'த': 37, 'd': 38, 'ு': 39, 'l': 40, 'o': 41, 'ா': 42, 'ற': 43, 'ண': 44, 'ல': 45, 'எ': 46, 'ஃ': 47, 'ி': 48, 'ௌ': 49, 'x': 50, 'ை': 51, 'j': 52, 'அ': 53, 'ூ': 54, 's': 55, 'ீ': 56, 'm': 57, 'ள': 58, 'ஸ': 59, 'ஏ': 60, 'க': 61, 'ொ': 62, 'ெ': 63, 'ழ': 64, 'உ': 65, 'q': 66, '்': 67, 'ஐ': 68, 'ஈ': 69, 'ங': 70, 'ப': 71}
